In [ ]:
import numpy as np
import networkx as nx
import time
import torch
import torch.nn.functional as F
import gc

        
# load dataset
testset = np.load('./testset_demo.npz')

test_features = torch.from_numpy(testset['F']/100.0)## normalized dBZ：(lgz)/10
test_lbs = torch.from_numpy( testset['L']) ## transformed precipitation labels：y=lg(y+1)
test_mask = torch.from_numpy(testset['M'])
del testset
gc.collect()

num_feats = test_features.shape[2]
N_test  = test_features.shape[0]
N_val = np.floor(N_test/2)
N_val = N_val.astype(np.int32)

trainset = np.load('./trainset_demo.npz') 
train_features = torch.from_numpy(trainset['F']/100.0)
train_lbs = torch.from_numpy( (trainset['L']) )
train_mask = torch.from_numpy(trainset['M'])
del trainset
gc.collect()

N_train  = train_features.shape[0]

indice = np.load('./rand_id_small.npz')
rand_id = indice['I'] ##indice for validation and testing examples


In [ ]:
train_features.shape

In [ ]:
###preprocessing the data for training###

test_labels=torch.zeros(test_lbs.shape)
train_labels=torch.zeros(train_lbs.shape)
k=1/6/np.log(10)
b=np.log10(6)-k*5
mask0=test_lbs<5
mask1=test_lbs>=5
C=torch.tensor([10.0]).float()

test_labels[mask0] = torch.log(test_lbs[mask0].float()+1)/torch.log(C)## transformed precipitation labels：y=0.1lg(R+1)
#test_labels[mask1] = (test_lbs[mask1]/9).float()
test_labels[mask1] = (k*test_lbs[mask1]+b).float()

mask0=train_lbs<5
mask1=train_lbs>=5
train_labels[mask0] = torch.log(train_lbs[mask0].float()+1)/torch.log(C)
train_labels[mask1] = (k*train_lbs[mask1]+b).float()

In [ ]:
import dgl
from dgl import DGLGraph
from dgl.data import register_data_args, load_data
#from gat import GAT
from snat3 import SNAT3
from utils import EarlyStopping


In [ ]:
import os

def mkdir(path): 
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)           

def lerelu(x):
    y=F.leaky_relu(x,negative_slope=0.1)
    return y
        
def mean_square_error(y,labels):
    mean_e= torch.sum((y-labels)**2)
    n = y.size(0)
    return mean_e, n

def b_ms_error(y,labels):
    s_e= torch.sum((labels+1)*(y-labels)**2)
    n = (labels+1).sum()
    return s_e, n

def b_abs_error(y,labels):
    s_e= torch.sum((labels+1)*abs(y-labels))
    n = (labels+1).sum()
    return s_e, n


def evaluate0(model, features, labels, mask, criterion):
    
    model.eval()
    with torch.no_grad():
        y = model(features)
        y = y[mask]
        y = y.flatten(0)
        labels = labels[mask] 
        L = labels#torch.zeros(y.shape)
        yy = y#torch.zeros(y.shape)
        mask0 = y<np.log10(6)
        mask1 = y>=np.log10(6)
        yy[mask0] = 10**(y[mask0])-1
        yy[mask1] = (y[mask1]-b)/k
        
        mask00 = labels<np.log10(6)
        mask11 = labels>=np.log10(6)
        L[mask00] = 10**(labels[mask00])-1
        L[mask11] = (labels[mask11]-b)/k
        return criterion(yy, L)


gpu= 0 #gpu device
#hyperparameters

threhold = 0.35##
num_heads=1
num_out_heads=1
num_layers = 6
num_hidden= 16
out_dim= 128
early_stop= True
fastmode= False

if gpu < 0:
    cuda = False
else:
    cuda = True
    torch.cuda.set_device(gpu)

from dgl.data.utils import load_graphs
g = load_graphs("./grid800_600.bin",[0])
g = g[0][0]
g = dgl.add_self_loop(g) 
if cuda:
    g = g.to(torch.device('cuda:%01d'%(gpu)))


# create model
heads = torch.zeros(num_layers+1,dtype=torch.int32)
nhidT = torch.zeros(num_layers+1,dtype=torch.int32)
heads[0:num_layers]=num_heads
heads[-1]=num_out_heads
nhidT[0]= 16
nhidT[1]=16
nhidT[2]=16
nhidT[3]=16
nhidT[4]=16
nhidT[5]=16
nhidT[6]=16
#nhidT[7]=16
#nhidT[8]=16
#nhidT[9]=16
#nhidT[10]=16
model = SNAT3(g,
             threhold,
            num_layers,
            num_feats,
            num_hidden,
            out_dim,
            heads=heads,
            num_hiddenT=nhidT,
            feat_drop =0,
            attn_drop = 0,
            negative_slope=0.2,
            activation= torch.tanh,##
            activationP= lerelu,
            residual=False)


if cuda:
    model.cuda()
    
print(model)
if early_stop:
    stopper = EarlyStopping(patience=50)

loss_fcn1 = torch.nn.MSELoss(reduction='none')   
loss_fcn2 = torch.nn.L1Loss(reduction='none')   

# use optimizer
optimizer1 = torch.optim.Adam(
    model.parameters(), lr=10e-6 )##
#optimizer1 = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

dur = []
for epoch in range(300):

    val_error=0
    loss_m = 0
    N_total = 0
    if epoch >= 3:
        t0 = time.time()
    rand_n=torch.randperm(N_train) 
    batchsize= 1  
    nbatches = int(N_train/batchsize/2) #use 1/2 of the training data every epoch
    N_nodes_train= 0
    for batch in range(nbatches):
        N_ba = 0
        for n in range(batchsize): 
            model.train()

            # forward
            index_n = rand_n[n+batchsize*batch]
            H0 = train_features[index_n][:,:].float()#.mean(dim=1).view(-1,1)
            mean_H0 = H0.mean(dim=1)
            #mean_H0 = H0
            index_0rd = (mean_H0==0).flatten()
            train_labels[index_n,index_0rd]=0##去除异常值
            index_2rd = (train_lbs[index_n]>100).flatten() ##把R>100 的都去掉
            train_mask[index_n,index_2rd]=False ##去除太大的zhi
            mask = train_mask[index_n].bool()
            labels = train_labels[index_n,mask].float()
            lbs = train_lbs[index_n,mask]
            if cuda== True:
                H0=H0.cuda()
                labels=labels.cuda()
                lbs = lbs.cuda()
            N_t = labels.size(0)
            y = model(H0) 
            index_0 = (labels) ==0 ###标签为0的索引
            W_nodes = lbs + 1#10**(labels)  ##  weigtht for none zero label: W
            #W_nodes[index_0]=0.01  ## weight for zero label: W0
            loss1 = loss_fcn1( y[mask,:].flatten(),
                            labels.float() )  ###compute the loss for every (labeled) node(MSE)
            loss2 = loss_fcn2( y[mask,:].flatten(),
                            labels.float() )  ###compute the loss for every (labeled) node(MAE)
            loss = loss1*W_nodes*0.1+ loss2*W_nodes
            loss_w = loss.mean() ## average loss among all nodes
            loss_m += loss.sum().item()
            loss_w.backward()
            N_ba +=N_t
        optimizer1.step() 
        optimizer1.zero_grad()
        N_nodes_train += N_ba           
    for n in range(N_val):
        model.eval()
        H0 = test_features[rand_id[n]][:,:].float()#.mean(dim=1).view(-1,1)
        mean_H0 = H0.mean(dim=1)
        index_0rd = (mean_H0==0).flatten()
        test_labels[rand_id[n],index_0rd]=0##去除异常值
        mask = (test_labels[rand_id[n]])>0
        index_2rd = (test_lbs[rand_id[n]]>100).flatten()
        test_mask[rand_id[n],index_2rd]=False ##去除太大的zhi
        mask = mask&test_mask[rand_id[n]].bool()
        test_F= H0
        test_L=test_labels[rand_id[n]].float()
        if cuda==True:
            test_F=test_F.cuda()
            test_L=test_L.cuda()
        val_error0, N0 = evaluate0(model, test_F, test_L, mask, b_abs_error)#
        val_error += val_error0
        N_total += N0
        
    val_error/=N_total
    
    path1='D:\\CNGATs\\CNGAT_parameters'
    mkdir(path1) 
    if early_stop: 
        if stopper.step(-val_error, model, path1+'\\demoCNGAT3_16_6_128_tanh_1frame_newgrid_wL1L2_01adam1e-5_th35_BMAE_MixedOutNewKB.pt'):   
            break
    loss_m = loss_m/N_nodes_train
    if epoch >= 3:
        dur.append(time.time() - t0)
    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.6f} |val_error{:.4f}".
              format(epoch, np.mean(dur), loss_m, val_error))
